# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd

In [ ]:
#| export
def move_columns(
    df: pd.DataFrame,  # Input
    cols_to_move: str,  # Single 
    pos: int  # Target
) -> pd.DataFrame:
    """
    Move one or more columns to a specified position in a DataFrame.
    """

    # Ensure `cols_to_move` is always a list
    if isinstance(cols_to_move, str):
        cols_to_move = [cols_to_move]

    cols = list(df.columns)

    # Remove the columns to be moved from the current order
    for c in cols_to_move:
        cols.remove(c)

    # Insert the removed columns after the specified position
    new_cols = cols[:pos] + cols_to_move + cols[pos:]

    # Return the reordered DataFrame
    return df[new_cols]


In [ ]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [ ]:
move_columns(df, 'C', 0)

,C,A,B
0,7,1,4
1,8,2,5
2,9,3,6


## Wrapper 

In [ ]:
a = 10

In [ ]:
def my_sum(b):
    return a + b

my_sum(32)

42

The idea is to write a wrapper that checks that the wrapped function depends only on the variables passed to it as argument. so sum should emit a warning or error because the function only works if we define a beforehand. is there already some tool doing this for me like a type checker of libary?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

There isn't a built-in tool that does exactly this, but there are a few approaches you could explore:

1. **Static analysis tools** like `pylint` can detect some cases of undefined variables, but they won't catch closure dependencies like your `sum` function using `a` from the outer scope.

2. **`inspect` module** - You can use `inspect.getclosurevars()` to detect what non-local variables a function depends on. This would let you build your own wrapper.

3. **Type checkers** like `mypy` focus on types rather than closure dependencies, so they won't help here.

The `inspect.getclosurevars()` approach seems most promising for what you want. Would you like to explore how to use that to build your wrapper, or would you prefer me to show you a working example?

In [ ]:
import inspect
# inspect.getclosurevars(eval("my_sum"))
inspect.getclosurevars(my_sum)


ClosureVars(nonlocals={}, globals={'a': 10}, builtins={}, unbound=set())

Say I want to disallow this in the whole jupyter notebook, I mean for all funcions defined in this notebook.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Nice find! You can see `globals={'a': 10}` which is exactly what you'd want to flag.

For notebook-wide enforcement, you could use an **IPython post-execution hook** — a function that runs automatically after every cell. It could inspect all functions in the namespace and warn about any that have non-empty `globals` in their closure vars.

The key pieces would be:
- `get_ipython().events.register('post_run_cell', your_checker)` to hook in
- Loop through items in the namespace, check which are functions
- Call `getclosurevars()` on each and flag those with `globals`

Want to try building it, or would you like me to sketch out the structure?

oh great. that sound like a good plan. how do the post-execution hooks works?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

IPython has an event system that lets you register callbacks for various lifecycle events. For post-cell execution, you use:

```python
def my_callback(result):
    print("Cell just finished running!")

get_ipython().events.register('post_run_cell', my_callback)
```

The `result` parameter is an `ExecutionResult` object containing info about what just ran (success/failure, etc.).

To unregister later:
```python
get_ipython().events.unregister('post_run_cell', my_callback)
```

Try registering a simple callback like the one above, then run a few cells to see it in action. Once you've got that working, you can expand it to do the closure checking.

In [ ]:
def my_callback(result):
    print("Cell just finished running!")

get_ipython().events.register('post_run_cell', my_callback)

Cell just finished running!


In [ ]:
get_ipython().events.unregister('post_run_cell', my_callback)

can I know if and which function was just defined in the running cell?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes! The `result` object has `result.info.raw_cell` which contains the source code of the cell that just ran. You can parse it with Python's `ast` module to find function definitions:

```python
import ast
tree = ast.parse(result.info.raw_cell)
```

Then look for `ast.FunctionDef` nodes in `tree.body`. Each one has a `.name` attribute with the function name.

Want to try extracting the function names from a parsed cell?🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes! A couple of approaches:

1. **From the result object**: `result.info.raw_cell` gives you the cell's source code. You could parse it with `ast` to find `def` statements.

2. **Snapshot comparison**: Store the set of functions before execution, then compare after to find new ones.

The `ast` approach is more precise since it tells you exactly what was *defined* in the cell, not just what's new in the namespace. Something like:

```python
import ast
tree = ast.parse(result.info.raw_cell)
func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
```

Which approach appeals to you?

I like approach 1 more. let's test it

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great choice! Try registering a callback that parses the cell and prints any function names it finds:

```python
import ast

def check_funcs(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    if func_names:
        print(f"Functions defined: {func_names}")

get_ipython().events.register('post_run_cell', check_funcs)
```

Then define a test function in another cell to see if it gets detected.

In [ ]:
import ast

def check_funcs(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    if func_names:
        print(f"Functions defined: {func_names}")

get_ipython().events.register('post_run_cell', check_funcs)

Functions defined: ['check_funcs']


In [ ]:
get_ipython().events.unregister('post_run_cell', check_funcs)

In [ ]:
def check_funcs(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    if func_names:
        print(f"Functions defined: {func_names}")

get_ipython().events.register('post_run_cell', check_funcs)

Functions defined: ['check_funcs']


In [ ]:
get_ipython().events.unregister('post_run_cell', check_funcs)

very nice. now let's put everything together

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Now you can combine the pieces:

1. Parse the cell to find function names
2. Look up each function in the namespace (`get_ipython().user_ns`)
3. Check `inspect.getclosurevars()` for non-empty `globals`
4. Warn if any are found

Want to give it a try, or would you like me to write it out?

show me how it works

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
import ast
import inspect

def check_global_deps(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    
    ns = get_ipython().user_ns
    for name in func_names:
        if name in ns:
            func = ns[name]
            cv = inspect.getclosurevars(func)
            if cv.globals:
                print(f"⚠️  '{name}' depends on global variables: {list(cv.globals.keys())}")

get_ipython().events.register('post_run_cell', check_global_deps)
```

Run this, then try defining your `my_sum` function again in a new cell to see if it catches the dependency on `a`.

In [ ]:
import ast
import inspect

def check_global_deps(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    
    ns = get_ipython().user_ns
    for name in func_names:
        if name in ns:
            func = ns[name]
            cv = inspect.getclosurevars(func)
            if cv.globals:
                print(f"⚠️  '{name}' depends on global variables: {list(cv.globals.keys())}")

get_ipython().events.register('post_run_cell', check_global_deps)

⚠️  'check_global_deps' depends on global variables: ['get_ipython', 'inspect', 'ast']


In [ ]:
get_ipython().events.unregister('post_run_cell', check_global_deps)

ah but this shows a false positive. modules and functions I would consider allright 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good catch! You can filter out modules and functions from the globals. Check the type of each value:

```python
import types

bad_globals = {k: v for k, v in cv.globals.items() 
               if not isinstance(v, (types.ModuleType, types.FunctionType, type))}
```

This keeps only "data" values and ignores modules, functions, and classes. Try updating your checker with this filter.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()